# Select top-N assets with good indicators
The objective is to calculate the standard indicators for a top N number of assets. Then use the indicators to decide on a potentially significant set of assets to consider for the portfolio. Thereafter, apply the MPT monte carlo algorithm to construct a weigted portfolio. 

In [1]:
'''
    WARNING CONTROL to display or ignore all warnings
'''
import warnings; warnings.simplefilter('ignore') #switch betweeb 'default' and 'ignore'
import traceback

''' Set debug flag to view extended error messages; 
    else set it to False to turn off debugging mode '''
debug = True

## Initialize the classes

In [2]:
import os
import sys
from datetime import datetime, date, timedelta
from pyspark.sql import functions as F

sys.path.insert(1,"/home/nuwan/workspace/rezaware/")
# import rezaware as reza
from mining.modules.assets.etp import dailyTopN as topN
from utils.modules.etl.transform import sparkCleanNRich as cnr

''' restart initiate classes '''
if debug:
    import importlib
#     reza = importlib.reload(reza)
    topN = importlib.reload(topN)

__desc__ = "analyze crypto market capitalization time series data"
clsTopN = topN.WeightedPortfolio(desc=__desc__)
clsCNR = cnr.Transformer(desc=__desc__)
print("\nClass initialization and load complete!")

All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
All functional SPARKCLEANNRICH-libraries in TRANSFORM-package of ETL-module imported successfully!
All functional DAILYTOPN-libraries in ETP-package of ASSETS-module imported successfully!
All functional APP-libraries in REZAWARE-package of REZAWARE-module imported successfully!
All functional SPARKDBWLS-libraries in LOAD-package of ETL-module imported successfully!
All functional SPARKNOSQLWLS-libraries in LOAD-package of ETL-module imported successfully!
sparkNoSQLwls Class initialization complete
All functional PERFORMINDEX-libraries in ETP-package of ASSETS-module imported successfully!
sparkNoSQLwls Class initialization complete
All packages in utils ml timeseries RollingStats imported successfully!
All functional EXECSESSION-libraries in SPARK-package of LIB-module imported successfully!
execSession Class initialization complete
performIndex Class initialization complete
All packages in util

## Select topN assets

### Read top N mcap assets from tip sql db

Set the following parameters to select the mcap data from the database
* ```_num_assets``` (integer) limits the number of asset count
* ```_mcap_val_lb```(decimal) limits the asset selection by mcap_value
* ```_date``` (datetime) selects assets with values for that day
* ```_table```(string) by default is 'warehouse.mcap_past' where mcap daily data is stored

Extends the ```utils/etl/load/sparkDBwls``` package to read the data from database table


### Construct a dict with selected assets
The dictionary serves as an input to the ```mining/modules/assets/etp/performIndex``` package to compuiting the index values.

In [3]:
_num_assets=3
_mcap_val_lb=10000.0
_date=datetime.strftime(date(2022,1,30),'%Y-%m-%dT00:00:00')
_table='warehouse.mcap_past'
kwargs={
    "TABLENAME":_table,
    "ASSETCOUNT":_num_assets,
    "STANDARDIZE":'minmax',
    "COLPREFIX":'sc',
    "WINLENGTH":7,
    "WINUNIT":'DAY',
}

# _data,_port=clsTopN.select_top_assets(
_features=clsTopN.select_top_assets(
    mcap_date=date(2022,1,30),
    mcap_value_lb=_mcap_val_lb,
    **kwargs,
)

23/03/29 23:18:55 WARN Utils: Your hostname, FarmRaiderTester resolves to a loopback address: 127.0.1.1; using 192.168.124.15 instead (on interface enp2s0)
23/03/29 23:18:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/03/29 23:18:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/29 23:18:56 WARN FileSystem: Cannot load filesystem: java.util.ServiceConfigurationError: org.apache.hadoop.fs.FileSystem: com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem Unable to get public no-arg constructor
23/03/29 23:18:56 WARN FileSystem: java.lang.NoClassDefFoundError: com/google/api/client/auth/oauth2/Credential
23/03/29 23:18:56 WARN FileSystem: java.lang.ClassNotFoundException: com.google.api.client.auth.oauth2.Credential


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/29 23:20:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


23/03/29 23:21:06 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/03/29 23:21:06 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK
[Error]function <indicators_wrapper> name 'clsCNR' is not defined


In [6]:
clsTopN.index.show()

23/03/29 23:22:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:22:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:22:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:22:50 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:22:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:22:51 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 2

23/03/29 23:22:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:22:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:22:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:22:55 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
+------------+--------------------+-----------+----------+------------+-------------------+--------------------+---------+--------------------+----------+--------------------+--------------------+-----------+--------------------+-------------+------------------+-------------------+--------------------+-------------------+
|mcap_past_pk|  

In [22]:
_data=clsCNR.scale(
    data=_features,
    col_list=['PCA[0]','PCA[1]'],
    col_prefix=kwargs['COLPREFIX'],
    scale_method=kwargs['STANDARDIZE'],
)
_data.show()

23/03/29 22:57:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 2

23/03/29 22:57:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/29 22:57:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:45 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 22:57:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 2

In [31]:
from pyspark.sql import functions as F

_data = _data.withColumn("1-PCA[0]",1-F.col('sc_PCA[0]'))\
            .withColumn("1-PCA[1]",1-F.col('sc_PCA[1]'))
_data.select(F.col('asset_name'),F.col('mcap_date'),
             F.col('1-PCA[0]'),F.col('1-PCA[1]'))\
                .sort(['1-PCA[0]','1-PCA[1]','asset_name'],ascending = False)\
                .show(n=30)

23/03/29 23:08:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:38 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 2

In [30]:
_data.select(F.col('asset_name'),F.col('mcap_date'),
             F.col('sc_PCA[0]'),F.col('sc_PCA[1]'))\
                .sort(['sc_PCA[0]','sc_PCA[1]','asset_name'],ascending = False)\
                .show(n=30)

23/03/29 23:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:32 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 23:08:33 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 2

In [226]:
'''DEPRECATED moved to dailyTopN class '''
_num_assets=23
_mcap_val_lb=10000.0
_date=datetime.strftime(date(2022,1,30),'%Y-%m-%dT00:00:00')
_table='warehouse.mcap_past'
kwargs={}

_query =f"select * from {_table} wmp where wmp.mcap_date = '{_date}' " +\
        f"and wmp.mcap_value > {_mcap_val_lb} " +\
        f"order by wmp.mcap_value DESC limit {_num_assets} "

_data = clsSDB.read_data_from_table(select=_query, **kwargs)
print("selected %d assets \n" % _data.count())
_data.show(n=1,vertical=True)

23/03/29 09:44:31 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


selected 23 assets 

-RECORD 0-----------------------------
 mcap_past_pk  | 19539                
 uuid          | 6397027b7cc473c58... 
 data_source   | coingecko            
 asset_name    | btc                  
 asset_symbol  | btc                  
 mcap_date     | 2022-01-30 00:00:00  
 mcap_value    | 729084397823.3950... 
 mcap_rank     | null                 
 created_dt    | 2023-02-14 12:07:... 
 created_by    | farmraider           
 created_proc  | wrangler_assets_e... 
 modified_dt   | 2023-02-21 14:45:... 
 modified_by   | farmraider           
 modified_proc | utils_etl_load_sp... 
 deactivate_dt | null                 
 log_ror       | 0.6966161566986901   
 simp_ror      | 0.0069020000000000   
only showing top 1 row



In [227]:
''' DEPRECATED moved to dailyTopN class '''
from pyspark.sql import functions as F
_assets=_data.select(F.col('mcap_date'),
                     F.col('asset_name'),
                     F.col('mcap_value'))\
                .distinct()
_portf=[]
for _asset in _assets.collect():
    _asset_dict={}
    _asset_dict={"date" : datetime.strftime(_asset[0],'%Y-%m-%dT00:00:00'),
                 "asset": _asset[1],
                 'mcap.weight': 1.0,
                 'mcap.value' : float(_asset[2]),
                }
    _portf.append(_asset_dict)
_portf=sorted(_portf, key=lambda d: d['mcap.value'], reverse=True)
_portf[:3]

[{'date': '2022-01-30T00:00:00',
  'asset': 'btc',
  'mcap.weight': 1.0,
  'mcap.value': 729084397823.395},
 {'date': '2022-01-30T00:00:00',
  'asset': 'bnb',
  'mcap.weight': 1.0,
  'mcap.value': 65455197964.9447},
 {'date': '2022-01-30T00:00:00',
  'asset': 'avax',
  'mcap.weight': 1.0,
  'mcap.value': 17939687507.8046}]

## Dimensionality reduction techniques
Review and select one
* Principal Component Analysis (PCA) - [sampling adequacy](https://statistics.laerd.com/spss-tutorials/principal-components-analysis-pca-using-spss-statistics.php)
* Generalized discriminant analysis (GDA)
* Missing Values Ratio.
* Low Variance Filter.
* [Feature selection](https://machinelearningmastery.com/feature-selection-with-real-and-categorical-data/) - unsupervised selection removing the target variable but dimensionality reduction is a replacement for unsupervised correlation based feature seelction.
* Feature extraction - not into finding isoltated signals in the data
* Non-negative matrix factorization (NMF) - we can normalize the data, which might be faster?
* Linear discriminant analysis (LDA) - it's a supervised technique


In [228]:
''' DEPRECATED moved to dailyTopN class '''
import pandas as pd

__idx_type__=['adx','sharp','rsi','mfi','beta']
_coll_dt=date(2022,1,30)
__val_col__="simp_ror"
__name_col__='asset_name'
__date_col__='mcap_date'
__rf_assets__=['btc']
__rf_val_col__="log_ror"
__rf_name_col__='asset_name'
__rf_date_col__='mcap_date'
_kwargs={
    "WINLENGTH":7,
    "WINUNIT":'DAY',
}
_res_df=pd.DataFrame()
# _results=[]
__idx_dict={}
for asset_portf in _portf:
    _idx_dict = clsIndx.get_index(
        portfolio=[asset_portf],
        asset_eval_date=_coll_dt,
        asset_name_col=__name_col__,
        asset_val_col =__val_col__,
        asset_date_col=__date_col__,
        index_type=__idx_type__,
        risk_free_assets=__rf_assets__,
        risk_free_name_col=__rf_name_col__,
        risk_free_val_col=__rf_val_col__,
        risk_free_date_col=__rf_date_col__,
        **_kwargs,
    )
    _idx_dict['asset']=asset_portf['asset']
    _res_df=pd.concat([_res_df,pd.DataFrame([_idx_dict])])
_res_df.insert(0, 'asset', _res_df.pop('asset'))
_res_df

,asset,adx,sharp,rsi,mfi,beta
0,btc,0.088397,36.016989,0.451438,0.451438,0.551231
0,bnb,0.320226,34.996164,0.292148,0.292148,0.580202
0,avax,0.386210,20.929375,0.468284,0.468284,-0.250196
0,matic,0.301415,18.348667,0.387615,0.387615,0.484944
0,cro,0.197762,23.021920,0.595720,0.595720,0.781872
0,wbtc,0.113276,50.618996,0.546493,0.546493,0.586002
0,atom,-0.333558,24.312151,0.203504,0.203504,0.546666
0,link,0.330922,18.270787,0.378757,0.378757,0.603306
0,near,0.176632,16.412486,0.268540,0.268540,0.558153
0,algo,0.443716,20.747471,0.266804,0.266804,0.105803


In [240]:
''' DEPRACATED moved to dailyTopN '''
from utils.modules.ml.dimreduc import pca

if debug:
    import importlib
    pca = importlib.reload(pca)
    
__desc__ = "process rolling stats for asset marketcap prices"

clsPCA = pca.FeatureEngineer(desc=__desc__)
print("\nClass initialization and load complete!")

All packages in utils ml dimreduc FeatureEngineer imported successfully!
execSession Class initialization complete

Class initialization and load complete!


In [ ]:
''' DEPRACATED moved to dailyTopN '''
from pyspark.sql import functions as F
from pyspark.sql import Window


kwargs={
    "STANDARDIZE":'minmax',
    "COLPREFIX":'sc'
}
_factors = clsPCA.get_features(
    data=_res_df,
#     part_col='asset'
    feature_cols=None,
    **kwargs,
)

In [246]:
'''DEPRECATED moved to dailyTopN class '''
_pca = _factors.withColumn("row_idx",
                                F.row_number()\
                                .over(Window.orderBy(F.monotonically_increasing_id())))
_data = clsPCA.data.withColumn("row_idx",
                               F.row_number()\
                               .over(Window.orderBy(F.monotonically_increasing_id())))

# _pca = _pca.join(_data.select(F.col('asset')),
#                            _pca.row_idx == _data.row_idx)
_data = _data.join(_pca, _pca.row_idx == _data.row_idx).drop('row_idx')
_pca=_pca.drop('row_idx')
_data.select(F.col('asset'),F.col('pca_features'),F.col('PCA[0]'),F.col('PCA[1]'))\
                .sort(['PCA[0]','PCA[1]','asset'])\
                .show()

23/03/29 11:35:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 11:35:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 11:35:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 11:35:03 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


23/03/29 11:35:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 11:35:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 11:35:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 11:35:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 11:35:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 11:35:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
23/03/29 1

+-----+--------------------+--------------------+--------------------+
|asset|        pca_features|              PCA[0]|              PCA[1]|
+-----+--------------------+--------------------+--------------------+
|  omi|[-1.5677300544611...| -1.5677300544611132|-0.07578867701870723|
| mana|[-1.1311689740290...| -1.1311689740290607| -0.3530275650262598|
|  cro|[-1.0353417852346...| -1.0353417852346398| -0.3337858256380067|
|  ftt|[-0.8576907710713...| -0.8576907710713784| -0.7258957622740481|
| wbtc|[-0.8124210730179...| -0.8124210730179725|-0.15771287388039856|
| avax|[-0.7654069361909...| -0.7654069361909335| -0.3538531612282307|
|  btc|[-0.6745011206326...| -0.6745011206326196|-0.30633683887131724|
|steth|[-0.6682474900436...| -0.6682474900436824| -0.3038743570869582|
|matic|[-0.6592571953428...| -0.6592571953428192| -0.5478265317171319|
| link|[-0.6588664138525...| -0.6588664138525273| -0.5937059850642774|
| hbar|[-0.6103415684669...| -0.6103415684669369| -0.5052446153843762|
|  axs